# CPB V1 vs V2 模型對比測試 - 單一筆記本执行

在 Colab 中扥一步執行，自動抽取 V1 訓練好的模型進行對比

## 架構
- WEB 端: cpb-trading-web
- 訓練端: cpbv2 (Colab) ← 當前
- 對比筆記本: 整整一個筆記本執行

## Cell 1: 安裝依賴 + 初始化

In [ ]:
# 安裝依賴
!pip install -q tensorflow numpy matplotlib pandas huggingface-hub requests

import os
import sys
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from datetime import datetime
import json
import urllib.request

# 設置中文字體
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print('[OK] 依賴安裝完成')

## Cell 2: 抽取貪數化 V1 模型截序檔

從 cpbv2 GitHub 下載已訓練好的 V1 模型預測函數

In [ ]:
print('[1] 抽取 V1 模型預測函數...')

import urllib.request

# 抽取並執行 V1 模型預測模組
try:
    urllib.request.urlretrieve(
        'https://raw.githubusercontent.com/caizongxun/cpbv2/main/notebooks/v1_model_predictor.py',
        'v1_predictor.py'
    )
    exec(open('v1_predictor.py').read())
    print('[OK] V1 模型預測模組已加載')
except Exception as e:
    print(f'[WARNING] 抽取失敗: {e}')
    print('[INFO] 使用底數歷章預測 (V1 版本)')
    
    # V1 底數預測模組 (皌後想抽取上面的模組)
    def predict_v1_hardcoded(current_price, direction):
        if direction == 'up':
            price_3h = current_price * 1.02
            price_5h = current_price * 1.03
        else:
            price_3h = current_price * 0.98
            price_5h = current_price * 0.97
        return price_3h, price_5h
    
    print('[OK] 使用底數 V1 預測函數')

## Cell 3: 抽取貪數化 V2 模型截序檔

從 cpbv2 GitHub 下載已訓練好的 V2 模型預測函數

In [ ]:
print('[2] 抽取 V2 模型預測函數...')

# 抽取並執行 V2 模型預測模組
try:
    urllib.request.urlretrieve(
        'https://raw.githubusercontent.com/caizongxun/cpbv2/main/notebooks/v2_model_predictor.py',
        'v2_predictor.py'
    )
    exec(open('v2_predictor.py').read())
    print('[OK] V2 模型預測模組已加載')
except Exception as e:
    print(f'[WARNING] 抽取失敗: {e}')
    print('[INFO] 使用動態波動率預測 (V2 版本)')
    
    # V2 模組 - 動態波動率
    def predict_v2_dynamic(current_price, direction, volatility, atr):
        atr_percent = (atr / current_price) * 100
        vol_factor = max(0.5, min(2.0, volatility / 2.0))
        dynamic_move_3h = atr_percent * vol_factor * 1.5
        dynamic_move_5h = atr_percent * vol_factor * 2.5
        
        if direction == 'up':
            price_3h = current_price * (1 + dynamic_move_3h / 100)
            price_5h = current_price * (1 + dynamic_move_5h / 100)
        else:
            price_3h = current_price * (1 - dynamic_move_3h / 100)
            price_5h = current_price * (1 - dynamic_move_5h / 100)
        
        return price_3h, price_5h, dynamic_move_3h, dynamic_move_5h
    
    print('[OK] 使用動態波動率 V2 預測函數')

## Cell 4: 抽取及定義輔助函數

從 cpbv2 GitHub 下載輔助函數 (ATR, 波動率, K線生成等)

In [ ]:
print('[3] 抽取輔助函數...')

try:
    urllib.request.urlretrieve(
        'https://raw.githubusercontent.com/caizongxun/cpbv2/main/notebooks/utility_functions.py',
        'utils.py'
    )
    exec(open('utils.py').read())
    print('[OK] 輔助函數已加載')
except Exception as e:
    print(f'[WARNING] 抽取失敗: {e}')
    print('[INFO] 使用本地定義的輔助函數')
    
    def calculate_atr(klines, period=14):
        if len(klines) < period:
            return 0
        trs = []
        for i in range(1, len(klines)):
            high_low = klines[i]['high'] - klines[i]['low']
            high_close = abs(klines[i]['high'] - klines[i-1]['close'])
            low_close = abs(klines[i]['low'] - klines[i-1]['close'])
            tr = max(high_low, high_close, low_close)
            trs.append(tr)
        atr = np.mean(trs[-period:]) if trs else 0
        return atr
    
    def calculate_volatility(klines, period=14):
        if len(klines) < period:
            return 0
        close_prices = [k['close'] for k in klines[-period:]]
        returns = np.diff(close_prices) / close_prices[:-1]
        volatility = np.std(returns) * 100
        return volatility
    
    def generate_historical_klines(num_candles=20, start_price=87800):
        klines = []
        current_price = start_price
        for i in range(num_candles):
            daily_return = np.random.normal(0, 0.015)
            open_price = current_price
            close_price = current_price * (1 + daily_return)
            high_price = max(open_price, close_price) * (1 + np.random.uniform(0, 0.01))
            low_price = min(open_price, close_price) * (1 - np.random.uniform(0, 0.01))
            klines.append({
                'time': i, 'open': open_price, 'high': high_price,
                'low': low_price, 'close': close_price, 'volume': np.random.uniform(1000, 5000)
            })
            current_price = close_price
        return klines
    
    def generate_future_klines(start_price, num_candles=5, true_direction=None):
        klines = []
        current_price = start_price
        for i in range(num_candles):
            if true_direction == 'down':
                daily_return = np.random.normal(-0.003, 0.008)
            elif true_direction == 'up':
                daily_return = np.random.normal(0.003, 0.008)
            else:
                daily_return = np.random.normal(0, 0.01)
            open_price = current_price
            close_price = current_price * (1 + daily_return)
            high_price = max(open_price, close_price) * (1 + np.random.uniform(0, 0.008))
            low_price = min(open_price, close_price) * (1 - np.random.uniform(0, 0.008))
            klines.append({
                'time': i, 'open': open_price, 'high': high_price,
                'low': low_price, 'close': close_price, 'volume': np.random.uniform(1000, 5000)
            })
            current_price = close_price
        return klines
    
    print('[OK] 使用本地輔助函數')

## Cell 5: 執行 V1 vs V2 對比測試

開始執行 20 次對比測試

In [ ]:
print('\n' + '='*70)
print('CPB Model V1 vs V2 Comparison Test')
print('='*70)

v1_errors = []
v2_errors = []
results = []

num_tests = 20

for test_num in range(num_tests):
    # Generate historical data
    hist_klines = generate_historical_klines(num_candles=20)
    current_price = hist_klines[-1]['close']
    
    # Calculate indicators
    atr = calculate_atr(hist_klines)
    volatility = calculate_volatility(hist_klines)
    direction = 'down' if np.random.random() > 0.5 else 'up'
    
    # V1 prediction
    v1_price_3h, v1_price_5h = predict_v1_hardcoded(current_price, direction)
    
    # V2 prediction
    v2_price_3h, v2_price_5h, move_3h, move_5h = predict_v2_dynamic(
        current_price, direction, volatility, atr
    )
    
    # Generate actual future klines
    future_klines = generate_future_klines(
        current_price,
        num_candles=5,
        true_direction=direction
    )
    actual_price_5h = future_klines[-1]['close']
    
    # Calculate errors
    v1_error = abs(v1_price_5h - actual_price_5h) / actual_price_5h * 100
    v2_error = abs(v2_price_5h - actual_price_5h) / actual_price_5h * 100
    
    v1_errors.append(v1_error)
    v2_errors.append(v2_error)
    
    results.append({
        'test': test_num + 1,
        'current_price': current_price,
        'volatility': volatility,
        'atr': atr,
        'direction': direction,
        'actual_5h': actual_price_5h,
        'v1_pred': v1_price_5h,
        'v2_pred': v2_price_5h,
        'v1_error': v1_error,
        'v2_error': v2_error
    })
    
    if (test_num + 1) % 5 == 0:
        print(f'Progress: [{test_num+1}/{num_tests}] V1 Error: {np.mean(v1_errors[-5:]):.2f}% | V2 Error: {np.mean(v2_errors[-5:]):.2f}%')

print('\n' + '='*70)
print('Test Complete')
print('='*70)

## Cell 6: 統計結果

In [ ]:
print('\n' + '='*70)
print('Overall Statistics')
print('='*70)

v1_mean = np.mean(v1_errors)
v2_mean = np.mean(v2_errors)
improvement = v1_mean - v2_mean
improvement_rate = (1 - v2_mean/v1_mean) * 100 if v1_mean > 0 else 0

print(f'\nV1 Average Error: {v1_mean:.2f}%')
print(f'V2 Average Error: {v2_mean:.2f}%')
print(f'Average Improvement: {improvement:+.2f}%')
print(f'Improvement Rate: {improvement_rate:+.1f}%')

print(f'\nV1 Std Dev: {np.std(v1_errors):.2f}%')
print(f'V2 Std Dev: {np.std(v2_errors):.2f}%')

print(f'\nV1 Min Error: {np.min(v1_errors):.2f}%')
print(f'V1 Max Error: {np.max(v1_errors):.2f}%')

print(f'V2 Min Error: {np.min(v2_errors):.2f}%')
print(f'V2 Max Error: {np.max(v2_errors):.2f}%')

v2_better_count = sum(1 for i in range(len(v1_errors)) if v2_errors[i] < v1_errors[i])
print(f'\nV2 Better Than V1: {v2_better_count}/{num_tests} times ({v2_better_count/num_tests*100:.1f}%)')

print('\n' + '='*70)

## Cell 7: 繪製對比圖表

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('CPB Model V1 vs V2 Comparison Analysis', fontsize=16, fontweight='bold')

# Error box plot
ax1 = axes[0, 0]
ax1.boxplot([v1_errors, v2_errors], labels=['V1 (Hardcoded)', 'V2 (Dynamic Volatility)'])
ax1.set_ylabel('Prediction Error (%)')
ax1.set_title('Error Distribution Comparison')
ax1.grid(True, alpha=0.3)

# Error trend
ax2 = axes[0, 1]
ax2.plot(range(1, len(v1_errors)+1), v1_errors, 'o-', label='V1', linewidth=2, markersize=4)
ax2.plot(range(1, len(v2_errors)+1), v2_errors, 's-', label='V2', linewidth=2, markersize=4)
ax2.set_xlabel('Test Number')
ax2.set_ylabel('Prediction Error (%)')
ax2.set_title('Error Trend')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Error histogram
ax3 = axes[1, 0]
ax3.hist(v1_errors, alpha=0.6, label='V1', bins=8)
ax3.hist(v2_errors, alpha=0.6, label='V2', bins=8)
ax3.set_xlabel('Prediction Error (%)')
ax3.set_ylabel('Frequency')
ax3.set_title('Error Distribution Histogram')
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')

# Improvement bar chart
ax4 = axes[1, 1]
improvements = [v1_errors[i] - v2_errors[i] for i in range(len(v1_errors))]
colors = ['green' if x > 0 else 'red' for x in improvements]
ax4.bar(range(1, len(improvements)+1), improvements, color=colors, alpha=0.7)
ax4.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax4.set_xlabel('Test Number')
ax4.set_ylabel('Improvement (%)')
ax4.set_title('V2 vs V1 Improvement (Positive=Better)')
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('model_comparison_v1_vs_v2.png', dpi=150, bbox_inches='tight')
print('Chart saved as: model_comparison_v1_vs_v2.png')

## Cell 8: 保存結果為 JSON

In [ ]:
with open('model_comparison_results.json', 'w') as f:
    json.dump({
        'test_date': datetime.now().isoformat(),
        'num_tests': num_tests,
        'v1_avg_error': float(np.mean(v1_errors)),
        'v2_avg_error': float(np.mean(v2_errors)),
        'improvement': float(improvement),
        'improvement_rate': float(improvement_rate),
        'v1_std': float(np.std(v1_errors)),
        'v2_std': float(np.std(v2_errors)),
        'v2_better_count': v2_better_count,
        'detailed_results': results
    }, f, indent=2)

print('[OK] Results saved to: model_comparison_results.json')

## Cell 9: 下載文件 (Colab)

In [ ]:
try:
    from google.colab import files
    print('[INFO] Downloading files...')
    files.download('model_comparison_results.json')
    files.download('model_comparison_v1_vs_v2.png')
    print('[OK] Files downloaded successfully')
except:
    print('[INFO] Not running in Colab, files saved locally')